In [56]:
# Dependencies and Setup
import pandas as pd
import requests
import json
import csv
import os
import gmaps
import pprint

#Load Google Places API key
from config import gkey

gmaps.configure(api_key=gkey)

# File to Load
cities = "cities.csv"

# Read city data file and store in Pandas Data frames
city_data = pd.read_csv(cities)
city_data.head()

,City,Country,Latitude,Longitude,Weather,Temperature,Humidity,Wind Speed,Cloud Cover %
0,Pevek,RU,69.70,170.31,Snow,0.57,92,7.58,100
1,Kavieng,PG,-2.57,150.80,Rain,83.53,73,3.83,9
2,Nikolskoye,RU,59.70,30.79,Clouds,44.22,68,6.71,100
3,Eisenerz,AT,47.53,14.88,Clear,43.00,75,2.24,0
4,Thompson,CA,55.74,-97.86,Clouds,32.00,95,10.29,75


In [101]:
# Extract the cities with ideal weather
ideal_temp = city_data[(city_data["Temperature"] >= 60) & (city_data["Temperature"] <= 80)]
no_wind = ideal_temp[ideal_temp["Wind Speed"] < 10]
clear_skys = no_wind[no_wind["Cloud Cover %"] < 10]
len(clear_skys)

37

In [65]:
#response_json = []
#name = []
#vicinity = []

#lat = 20.52
#lng = -13.05

# geocoordinates
#target_coordinates = f"{lat},{lng}"
#target_search = "Hotel"
#target_radius = 5000
#target_type = "hotel"

# set up a parameters dictionary
#params = {
#    "location": target_coordinates,
#    "keyword": target_search,
#    "radius": target_radius,
#    "type": target_type,
#    "key": gkey
#}

#base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary
#response_json = requests.get(base_url, params=params).json()

#try:
#    name.append(response_json["results"][0]["name"])
#    vicinity.append(response_json["results"][0]["vicinity"])

#response_json.append(response)

#except KeyError: 
#    print("Cannot find city")
    

        
#response_json

In [97]:
response_json = []
name = []
vicinity = []
coords = []


for index, row in clear_skys.iterrows():
    lat = row["Latitude"]
    lng = row["Longitude"]
    

    # geocoordinates
    target_coordinates = f"{lat},{lng}"
    print(lat)
    print(lng)
    target_search = "Hotel"
    target_radius = 5000
    target_type = "hotel"

    # set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": gkey
    }

    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    response_json = requests.get(base_url, params=params).json()
    
    try:
        coords.append(target_coordinates)
        name.append(response_json["results"][0]["name"])
        vicinity.append(response_json["results"][0]["vicinity"])
        
    #response_json.append(response)
    
    except KeyError: 
        print("Cannot find city")
    except IndexError:
        print("No results")
        name.append("No results")
        vicinity.append("No results")

-14.27
50.17
-21.36
55.77
25.52
90.22
-21.37
55.62
27.04
14.43
39.55
-77.02
20.35
-74.5
-38.95
-68.06
38.52
-28.7
17.49
44.13
No results
-28.57
-70.76
42.8
93.45
18.03
73.02
3.52
39.06
-41.15
-71.31
-15.2
12.15
26.59
12.78
No results
-21.34
55.48
-23.49
-58.8
-38.27
141.97
25.26
63.47
13.35
-15.75
No results
-41.87
-73.82
33.19
-82.53
No results
-23.68
22.79
-31.9
-61.86
20.2
1.01
No results
-18.77
46.03
19.88
75.33
18.42
84.05
39.6
-110.81
-13.68
48.45
8.82
40.52
41.76
86.15
-22.4
46.12
-20.28
44.28
-23.42
-54.65
No results


In [103]:
len(name)

37

In [61]:
# convert response to json
#response = response.json()

# Print the json (pretty printed)
print(json.dumps(response_json, indent=4, sort_keys=True))

{
    "html_attributions": [],
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -14.2579014,
                    "lng": 50.1605447
                },
                "viewport": {
                    "northeast": {
                        "lat": -14.25669437010728,
                        "lng": 50.16195182989272
                    },
                    "southwest": {
                        "lat": -14.25939402989272,
                        "lng": 50.15925217010728
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "id": "cb2bd131f1824d569f3327651ffd1342e63b87c3",
            "name": "Carrefour",
            "photos": [
                {
                    "height": 3072,
                    "html_attributions": [
                        "<a href=\"https://maps.google.com/maps/contrib/110585159040346560607\">Rhonda Robe

In [109]:
# convert response to json
#places_data = response.json()

# augment the google api data to the clear_skys dataframe

clear_skys["Hotel Vicinity"] = vicinity
clear_skys["Hotel Name"] = name

clear_skys

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,City,Country,Latitude,Longitude,Weather,Temperature,Humidity,Wind Speed,Cloud Cover %,Hotel Name,Hotel Vicinity
61,Sambava,MG,-14.27,50.17,Rain,74.86,82,9.22,2,Carrefour,Sambava
85,Saint-Philippe,RE,-21.36,55.77,Clear,68.76,73,8.05,4,Pinpin d'Amour,"56 A Chemin Paul Hoareau, Baril les Hauts"
104,Tura,IN,25.52,90.22,Clear,68.32,57,2.48,3,Hotel Polo Orchid,"Araimile, Tura"
118,Saint-Joseph,RE,-21.37,55.62,Clear,68.81,73,8.05,2,Les Lodges Vanilliers,Saint-Joseph
125,Sabha,LY,27.04,14.43,Clear,61.18,32,9.24,4,فندق المواجيد,"مخازن الأسواق سابقاً, سبها-حي عبدالكافي مقابل,..."
157,Carroll,US,39.55,-77.02,Clear,71.02,46,6.93,1,Best Western Westminster Hotel,"451 WMC Dr, Westminster"
192,Baracoa,CU,20.35,-74.50,Clear,77.25,77,8.79,1,El Castillo,Baracoa
208,Neuquén,AR,-38.95,-68.06,Clear,73.40,35,6.93,0,Neuquen Tower Hotel,"Gral. Manuel Belgrano 174, Neuquén"
226,Ribeira Grande,PT,38.52,-28.70,Clear,60.71,84,8.16,6,Quinta Das Buganvílias,Castelo Branco
260,Najrān,SA,17.49,44.13,Clear,75.20,14,1.99,0,No results,No results


In [63]:
locations = city_data[["Latitude", "Longitude"]]
humidity = city_data["Humidity"]

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, 
                                 max_intensity=10, point_radius=1)

fig.add_layer(heat_layer)

fig


Figure(layout=FigureLayout(height='420px'))

In [64]:
locations = clear_skys[["Latitude", "Longitude"]]
humidity = clear_skys["Humidity"]

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, 
                                 max_intensity=10, point_radius=1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [126]:
locations = clear_skys[["Latitude", "Longitude"]]
humidity = clear_skys["Humidity"]

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, 
                                 max_intensity=10, point_radius=1)

hotel_names = clear_skys["Hotel Name"]
hotel_vicinity = clear_skys["Hotel Vicinity"]

hotel_info = clear_skys["Hotel Name"]

info_box_template = """
<dl>
<dt>Name</dt><dd>{name}</dd>
<dt>Hotel vicinity</dt><dd>{vicinity}</dd>
</dl>
"""

plant_info = [info_box_template.format(**plant) for plant in nuclear_power_plants]

marker_layer = gmaps.marker_layer(locations, info_box_content=info_box_template)

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
fig


Figure(layout=FigureLayout(height='420px'))

In [118]:
clear_skys.head()

,City,Country,Latitude,Longitude,Weather,Temperature,Humidity,Wind Speed,Cloud Cover %,Hotel Name,Hotel Vicinity
61,Sambava,MG,-14.27,50.17,Rain,74.86,82,9.22,2,Carrefour,Sambava
85,Saint-Philippe,RE,-21.36,55.77,Clear,68.76,73,8.05,4,Pinpin d'Amour,"56 A Chemin Paul Hoareau, Baril les Hauts"
104,Tura,IN,25.52,90.22,Clear,68.32,57,2.48,3,Hotel Polo Orchid,"Araimile, Tura"
118,Saint-Joseph,RE,-21.37,55.62,Clear,68.81,73,8.05,2,Les Lodges Vanilliers,Saint-Joseph
125,Sabha,LY,27.04,14.43,Clear,61.18,32,9.24,4,فندق المواجيد,"مخازن الأسواق سابقاً, سبها-حي عبدالكافي مقابل,..."
